#### Importing Libs

In [1]:
import sqlalchemy as sql
import pandas as pd
from dotenv import load_dotenv
import os
import glob
import re
from slugify import slugify
import numpy as np
import time
from SQLeto import SQLeto

#### Loading enviroment vars

In [2]:
load_dotenv('..\.env')
DATABASE_EXERCISE = os.getenv('DATABASE_EXERCISE')
USER_EXERCISE = os.getenv('USER_EXERCISE')
PWD_EXERCISE = os.getenv('PWD_EXERCISE')
TABLES_PATH = os.getenv('TABLES_PATH')
PROCESSED_DATA_PATH = os.getenv('PROCESSED_DATA_PATH')

#### Load dataframes

In [3]:
# load .csv of companies
df_companies = pd.read_csv(
    TABLES_PATH+'companies.csv', 
    parse_dates=['FOUNDING_DATE'])
        
# load .csv of people
df_people = pd.read_csv(
    TABLES_PATH+'people.csv', 
    parse_dates=[
        'GROUP_START_DATE',
        'GROUP_END_DATE'])

In [4]:
# Preview of data companies
display(df_companies.head()), df_companies.info()

,NAME,COMPANY_LINKEDIN_NAMES,DESCRIPTION,HEADCOUNT,FOUNDING_DATE,MOST_RECENT_RAISE,MOST_RECENT_VALUATION,INVESTORS,KNOWN_TOTAL_FUNDING
0,ORSYP,"[\n ""orsyp""\n]",IT Operations Management Specialists At ORSYP ...,63.0,1986-01-01,NaN,NaN,NaN,NaN
1,Runwal,"[\n ""runwal""\n]",The Runwal Group was established in 1978 by it...,406.0,1978-01-01,NaN,NaN,"[\n ""Brand Capital""\n]",NaN
2,Toast,"[\n ""toast-inc""\n]",Toast empowers restaurants of all sizes to bui...,3580.0,2011-12-22,400000000.0,4.900000e+09,"[\n ""Eight Roads Ventures"",\n ""Greenoaks Cap...",899000000.0
3,DNA Medicine Institute,"[\n ""dna-medicine-institute""\n]","The DNA Medicine Institute, Inc. (DMI) is a me...",2.0,2004-01-01,NaN,NaN,NaN,NaN
4,Ally,"[\n ""ally""\n]",Ally Financial Inc. (NYSE: ALLY) is a leading ...,12120.0,1919-01-01,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10711 entries, 0 to 10710
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   NAME                    10711 non-null  object 
 1   COMPANY_LINKEDIN_NAMES  10711 non-null  object 
 2   DESCRIPTION             10679 non-null  object 
 3   HEADCOUNT               6283 non-null   float64
 4   FOUNDING_DATE           9230 non-null   object 
 5   MOST_RECENT_RAISE       2110 non-null   float64
 6   MOST_RECENT_VALUATION   419 non-null    float64
 7   INVESTORS               3139 non-null   object 
 8   KNOWN_TOTAL_FUNDING     2349 non-null   float64
dtypes: float64(4), object(5)
memory usage: 753.2+ KB


(None, None)

In [5]:
# Preview of data people
display(df_people.head()),df_people.info()

,PERSON_ID,COMPANY_NAME,COMPANY_LI_NAME,LAST_TITLE,GROUP_START_DATE,GROUP_END_DATE
0,9fb750ce-4acd-40d6-a58b-f6718342364f,GoCardless,gocardless,Software Engineer,2019-01-01,2020-01-01
1,9fb750ce-4acd-40d6-a58b-f6718342364f,Stealth startup,online-shoe-store,Founder / CTO,2018-01-01,2019-01-01
2,9fb750ce-4acd-40d6-a58b-f6718342364f,Arkera,arkera,Software Engineer,2017-01-01,2018-01-01
3,9fb750ce-4acd-40d6-a58b-f6718342364f,Imperial College London,imperial-college-london,UTA (Undergraduate Teaching Assistant),2016-01-01,2017-01-01
4,15f5d8ed-36ad-4cf7-8748-c50dc9589f59,Splunk,splunk,Software Engineer,2019-10-01,NaT


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5391 entries, 0 to 5390
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   PERSON_ID         5391 non-null   object        
 1   COMPANY_NAME      5390 non-null   object        
 2   COMPANY_LI_NAME   4471 non-null   object        
 3   LAST_TITLE        5391 non-null   object        
 4   GROUP_START_DATE  5348 non-null   datetime64[ns]
 5   GROUP_END_DATE    4187 non-null   datetime64[ns]
dtypes: datetime64[ns](2), object(4)
memory usage: 252.8+ KB


(None, None)

#### Processing data

In [6]:
# Dropping line with no company name
df_people = df_people.drop(
    df_people[
        df_people['COMPANY_NAME'].isna()].index,axis=0).reset_index(
            drop=True)

In [7]:
# Treating column COMPANY_LINKEDIN_NAMES
df_companies['COMPANY_LINKEDIN_NAMES'] = df_companies.COMPANY_LINKEDIN_NAMES.apply(lambda x: eval(x))
df_companies['COMPANY_LINKEDIN_NAMES'] = df_companies.COMPANY_LINKEDIN_NAMES.apply(lambda x: x[0] if len(x) > 0 else np.NaN)
# Treating column INVESTORS
df_companies['INVESTORS'] = df_companies.INVESTORS.fillna('[]').apply(lambda x: eval(x))
df_companies['INVESTORS'] = df_companies.INVESTORS.apply(lambda x: ', '.join(x))

In [8]:
df_companies.head()

,NAME,COMPANY_LINKEDIN_NAMES,DESCRIPTION,HEADCOUNT,FOUNDING_DATE,MOST_RECENT_RAISE,MOST_RECENT_VALUATION,INVESTORS,KNOWN_TOTAL_FUNDING
0,ORSYP,orsyp,IT Operations Management Specialists At ORSYP ...,63.0,1986-01-01,NaN,NaN,,NaN
1,Runwal,runwal,The Runwal Group was established in 1978 by it...,406.0,1978-01-01,NaN,NaN,Brand Capital,NaN
2,Toast,toast-inc,Toast empowers restaurants of all sizes to bui...,3580.0,2011-12-22,400000000.0,4.900000e+09,"Eight Roads Ventures, Greenoaks Capital, T. Ro...",899000000.0
3,DNA Medicine Institute,dna-medicine-institute,"The DNA Medicine Institute, Inc. (DMI) is a me...",2.0,2004-01-01,NaN,NaN,,NaN
4,Ally,ally,Ally Financial Inc. (NYSE: ALLY) is a leading ...,12120.0,1919-01-01,NaN,NaN,,NaN


In [9]:
# Construct linkedin names to empty data in df_companies
index = df_companies.loc[df_companies['COMPANY_LINKEDIN_NAMES'].isna()].index
for idx in index :
    df_companies.loc[idx, 'COMPANY_LINKEDIN_NAMES'] = slugify(df_companies.loc[idx]['NAME'])

In [10]:
# Construct linkedin names to empty data in df_companies
index = df_people.loc[df_people['COMPANY_LI_NAME'].isna()].index
for idx in index :
    df_people.loc[idx, 'COMPANY_LI_NAME'] = slugify(df_people.loc[idx]['COMPANY_NAME'])

In [11]:
df_companies = df_companies.reset_index().rename(columns={'index':'company_id'})
df_people = df_people.reset_index().rename(columns={'index':'people_id'})

In [12]:
# Verifying number of intersection between tables, using with reference Linkedin names in both tables.
print(f'''Interseções entre as tabelas : {
    len(
        df_companies[
            df_companies.COMPANY_LINKEDIN_NAMES.isin(
                df_people.COMPANY_LI_NAME)])}''')

Interseções entre as tabelas : 1717


#### Instance Object SQLeto

In [13]:
sqleto = SQLeto(DATABASE_EXERCISE, USER_EXERCISE, PWD_EXERCISE)

In [14]:
sqleto.execute_DDL(
    ''' DROP TABLE IF EXISTS companies CASCADE;
        DROP TABLE IF EXISTS people CASCADE;
    ''')

In [15]:
sqleto.execute_DDL(
    sqleto.creating_tables(df_companies, 'companies', 'company_id'))

In [16]:
sqleto.creating_tables(df_companies, 'companies', 'company_id')

'CREATE TABLE companies(company_id INTEGER,NAME VARCHAR(178),COMPANY_LINKEDIN_NAMES VARCHAR(203),DESCRIPTION TEXT,HEADCOUNT REAL,FOUNDING_DATE VARCHAR(60),MOST_RECENT_RAISE REAL,MOST_RECENT_VALUATION REAL,INVESTORS TEXT,KNOWN_TOTAL_FUNDING REAL,PRIMARY KEY (company_id));'

In [17]:
sqleto.execute_DDL(
    sqleto.creating_tables(
        dataframe=df_people,
        name_table='people',
        primary_key='people_id'))

In [18]:
df_companies.columns = [c.lower() for c in df_companies.columns]
df_people.columns = [c.lower() for c in df_people.columns]

#### Save dataframes in processed data

In [19]:
df_companies.to_csv(PROCESSED_DATA_PATH+'companies.csv', index=False)
df_people.to_csv(PROCESSED_DATA_PATH+'people.csv', index=False)

#### Adding data to PostgreSQL

In [20]:
sqleto.send_dataframes_to_tables(PROCESSED_DATA_PATH, if_exists_='append')

Saving table companies
Table companies saved!

Saving table people
Table people saved!



#### Verify data uploaded

In [21]:
sqleto.execute_DQL(
    '''SELECT * from companies''').head()

,company_id,name,company_linkedin_names,description,headcount,founding_date,most_recent_raise,most_recent_valuation,investors,known_total_funding
0,0,ORSYP,orsyp,IT Operations Management Specialists At ORSYP ...,63.0,1986-01-01,NaN,NaN,None,NaN
1,1,Runwal,runwal,The Runwal Group was established in 1978 by it...,406.0,1978-01-01,NaN,NaN,Brand Capital,NaN
2,2,Toast,toast-inc,Toast empowers restaurants of all sizes to bui...,3580.0,2011-12-22,400000000.0,4.900000e+09,"Eight Roads Ventures, Greenoaks Capital, T. Ro...",899000000.0
3,3,DNA Medicine Institute,dna-medicine-institute,"The DNA Medicine Institute, Inc. (DMI) is a me...",2.0,2004-01-01,NaN,NaN,None,NaN
4,4,Ally,ally,Ally Financial Inc. (NYSE: ALLY) is a leading ...,12120.0,1919-01-01,NaN,NaN,None,NaN


In [22]:
sqleto.execute_DQL(
    '''SELECT * from people''').head()

,people_id,person_id,company_name,company_li_name,last_title,group_start_date,group_end_date
0,0,9fb750ce-4acd-40d6-a58b-f6718342364f,GoCardless,gocardless,Software Engineer,2019-01-01,2020-01-01
1,1,9fb750ce-4acd-40d6-a58b-f6718342364f,Stealth startup,online-shoe-store,Founder / CTO,2018-01-01,2019-01-01
2,2,9fb750ce-4acd-40d6-a58b-f6718342364f,Arkera,arkera,Software Engineer,2017-01-01,2018-01-01
3,3,9fb750ce-4acd-40d6-a58b-f6718342364f,Imperial College London,imperial-college-london,UTA (Undergraduate Teaching Assistant),2016-01-01,2017-01-01
4,4,15f5d8ed-36ad-4cf7-8748-c50dc9589f59,Splunk,splunk,Software Engineer,2019-10-01,None
